# Arviz Data Structures

While ArviZ supports plotting from familiar datatypes, such as dictionaries and numpy arrays, there are a couple data structures central to ArviZ that are useful to know when using the library. 

They are
* xarray
* InferenceData
* netcdf


## Why more than one data structure?
Bayesian Inference generates numerous sets that represent different aspects of the model. For example in a single analysis a bayesian practioner could end up with any of the following data.
* Prior Distribution for N number of variables
* Posterior Distribution for N number of variables
* Prior Predictive Distribution
* Posterior Predictive Distribution
* Trace data for each of the above
* Sample statistics for each inference run
* Whatever else

## Why not Pandas Dataframes or Numpy Arrays?
Pandas would be a wonderful data structure, but data from probabilistic programming is naturally high dimensional. that is precisely the niche xarray seeks to fill To add to the complexity ArviZ must handle the data generated from multiple Bayesian Modeling libraries, such as pymc3 and pystan.

Although seemingly more complex at a glance the Arviz devs believe that the usage of *xarray*, *InferenceData*, and *netcdf* will simply the handling, referencing, and serialization of data generated by MCMC runs.

## An introduction to each
To help get familiar with each ArviZ includes some toy datasets. To start an `az.InferenceData` sample can be loaded into memory quite easily.

In [2]:
# Load the centered eight schools model
import arviz as az
data = az.load_arviz_data('centered_eight')
data

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data

In this case the `az.InferenceData` object contains both a posterior_predictive distribution, and the observed data, among other datasets. Each group in InferenceData is both an attribute on `InferenceData` and itself an `xarray.DataSet` object. 

In [3]:
# Get the posterior xarray
posterior = data.posterior
posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * school   (school) object 'Choate' 'Deerfield' 'Phillips Andover' ...
Data variables:
    mu       (chain, draw) float64 ...
    theta    (chain, draw, school) float64 ...
    tau      (chain, draw) float64 ...

In our eight schools example it can see that this particular posterior trace consists of three variables, estimated over 4 chains. In addition this model is a hierachial models where values for the variable `theta` are associated with a particular school. 

In xarray terminology Data Variable are the actual values generated from the MCMC draws, Dimensions are the axes on which can refer to the Data Variables and Coordinates which are pointers to specific slices or points in the `xarray.DataSet`

Observed data can be accessed through the same method.

In [4]:
# Get the observed xarray
observed_data = data.observed_data
observed_data

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' 'Phillips Andover' ...
Data variables:
    obs      (school) float64 ...

It should be noted that the observed dataset contains only 8 DataVariables and shares no dimensions or coordinates with the posterior. This difference in sizes is the motivating reason behind *InferenceData*. Rather than force multiple different sized arrays into one array, or force users to manage multiple objects corresponding to different datasets, it is easier to hold references to each *xarray* in an *InferenceData* object.

## NetCDF
[NetCDF](https://www.unidata.ucar.edu/software/netcdf/) is a standard for referencing array oriented files. In other words while, *xarray.Dataset*s, and by extension *InferenceData*, are convenient for accessing arrays in Python memory, *NetCDF* provides a convenient mechanism for persistence of model data on disk.

Most users will not have to concern themselves with the *netcdf* standard but for completeness it is good to make its usage transparent.

Earlier in this tutorial *InferenceData* was loaded from a *NetCDF* file

In [5]:
data = az.load_arviz_data('centered_eight')

Similarly the *InferenceData* objects can be persisted tp disk in the NetCDF format

In [6]:
data.to_netcdf("eight_schools_model.nc")

'eight_schools_model.nc'

# Additional Reading
Additional documentation and tutorials exist for xarray and netcd4.

## xarray
* [xarray documentation](http://xarray.pydata.org/en/stable/why-xarray.html)
* [xarray lightning talk at scipy 2015](https://www.youtube.com/watch?v=X0pAhJgySxk&t=949s)

## netcdf
* [netcd documentation](http://unidata.github.io/netcdf4-python/)
* [netcd usage in xarray](http://xarray.pydata.org/en/stable/io.html#netcdf)